In [61]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

# --- CONFIGURATION ---
# Use raw strings (r"...") to avoid path errors
DATASET_PATH = r"/content/drive/MyDrive/aakash DS/datasets/Jute_Pest_Dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# 1. LOAD DATA EFFICIENTLY
# We use image_dataset_from_directory which is faster and cleaner than ImageDataGenerator
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATASET_PATH, "train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATASET_PATH, "val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATASET_PATH, "test"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
)

Found 3805 files belonging to 17 classes.
Found 413 files belonging to 17 classes.
Found 379 files belonging to 17 classes.


In [62]:
# Save class names for later use (Prediction)
class_names = train_ds.class_names
print(f"Detected {len(class_names)} Classes: {class_names}")

# Optimize performance (keeps data ready in memory)
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Detected 17 Classes: ['Beet Armyworm', 'Black Hairy', 'Cutworm', 'Field Cricket', 'Jute Aphid', 'Jute Hairy', 'Jute Red Mite', 'Jute Semilooper', 'Jute Stem Girdler', 'Jute Stem Weevil', 'Leaf Beetle', 'Mealybug', 'Pod Borer', 'Scopula Emissaria', 'Termite', 'Termite odontotermes (Rambur)', 'Yellow Mite']


In [63]:

# 2. BUILD MODEL (TRANSFER LEARNING)
# We use MobileNetV2 pre-trained on ImageNet
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False, # We want our own output layer, not the original 1000 classes
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model initially

model = models.Sequential([
    # Input layer
    layers.Input(shape=(224, 224, 3)),

    # Data Augmentation (Helps if dataset is small)
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),

    # Preprocessing (MobileNet expects -1 to 1 scaling)
    layers.Rescaling(1./127.5, offset=-1),

    # The Pre-trained Base
    base_model,

    # The Classifier Head
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation='softmax')
])


# model = models.Sequential([
#     layers.Input(shape=(224,224,3)),

#     layers.Rescaling(1./255),

#     # Block 1
#     layers.Conv2D(32, 3, padding="same", activation="relu"),
#     layers.Conv2D(32, 3, padding="same", activation="relu"),
#     layers.MaxPooling2D(),

#     # Block 2
#     layers.Conv2D(64, 3, padding="same", activation="relu"),
#     layers.Conv2D(64, 3, padding="same", activation="relu"),
#     layers.MaxPooling2D(),

#     # Block 3
#     layers.Conv2D(128, 3, padding="same", activation="relu"),
#     layers.Conv2D(128, 3, padding="same", activation="relu"),
#     layers.MaxPooling2D(),

#     # Block 4
#     layers.Conv2D(256, 3, padding="same", activation="relu"),
#     layers.Conv2D(256, 3, padding="same", activation="relu"),
#     layers.MaxPooling2D(),

#     layers.GlobalAveragePooling2D(),

#     layers.Dense(256, activation="relu"),
#     layers.Dropout(0.5),
#     layers.Dense(len(class_names), activation="softmax")
# ])


In [64]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [65]:
# 3. TRAIN
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10  # You can increase this if accuracy is still going up
)

# 4. EVALUATE & SAVE
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 28s 185ms/step - accuracy: 0.5140 - loss: 1.5402 - val_accuracy: 0.6150 - val_loss: 2.8051
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.8916 - loss: 0.3515 - val_accuracy: 0.6538 - val_loss: 2.8508
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.9246 - loss: 0.2384 - val_accuracy: 0.6780 - val_loss: 2.9201
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.9493 - loss: 0.1779 - val_accuracy: 0.6755 - val_loss: 3.0021
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9597 - loss: 0.1486 - val_accuracy: 0.6973 - val_loss: 3.0457
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9729 - loss: 0.1145 - val_accuracy: 0.6973 - val_loss: 3.0943
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9673 - loss: 0.1132 - val_accuracy: 0.6949 - val_loss: 3.1413
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.9714 - loss: 0.1059 - val_ac

In [66]:
model.evaluate(test_ds)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.7344 - loss: 2.9116


[4.857093811035156, 0.5672823190689087]

In [ ]:
# # 1. Unfreeze part of MobileNet
# base_model.trainable = True

# fine_tune_at = int(len(base_model.layers) * 0.7)
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False

# # 2. Recompile with LOWER LR (REPLACES previous compile)
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-5),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# # 3. Train again
# history_finetune = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=10
# )